# Домашнее задание. Нейросетевая классификация текстов

В этом домашнем задании вам предстоит самостоятельно решить задачу классификации текстов. Вы будете классифицировать тксты писателей по авторству. Всего 8 разных авторов.

Скачаем датасет:

In [ ]:
!wget -O train_authors.csv https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/train_authors_new.csv

--2024-11-26 18:25:14--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/train_authors_new.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5163531 (4.9M) [text/plain]
Saving to: ‘train_authors.csv’

train_authors.csv   100%[===================>]   4.92M  --.-KB/s    in 0.1s    

2024-11-26 18:25:14 (47.9 MB/s) - ‘train_authors.csv’ saved [5163531/5163531]



In [ ]:
!wget -O test_authors.csv https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/test_authors_new.csv

--2024-11-26 18:25:17--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/test_authors_new.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 961703 (939K) [text/plain]
Saving to: ‘test_authors.csv’

test_authors.csv    100%[===================>] 939.16K  --.-KB/s    in 0.06s   

2024-11-26 18:25:18 (15.9 MB/s) - ‘test_authors.csv’ saved [961703/961703]



Импортируем необходимые библиотеки:

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
from tqdm import tqdm_notebook
import torch.nn as nn                                                             # тут все блоки нейронных сетей, слои
import torch.nn.functional as F

from collections import Counter
from typing import List
import string

import seaborn
seaborn.set(palette='summer')

# скачиваем нужный пакет данных для работы библиотеки nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Загрузим даатсет и посмотрим на данные:

In [ ]:
# Загрузим датасет
import pandas as pd
train_data = pd.read_csv('train_authors.csv')
train_data.head()

,text,label
0,"-Да, я поторопился. Капитан, примите мои извин...",Pratchett
1,-Похороны по первому разряду! Довольно благоро...,Remark
2,"Третий округ штата Мэн настолько велик, что ег...",King
3,В мире существуют миллиарды и миллиарды богов....,Pratchett
4,Особенность историографии киевского периода со...,Akunin


Здесь поле 'text' — это текст, а 'label' — автор текста. 'label' — это целевая переменная, которую мы будем предсказывать. Обратите внимания, что в тестовых данных столбца 'label' нет, вам нужно будет предсказать его значения и отправить в качестве ответа на Яндекс.Контест.

In [ ]:
test_data = pd.read_csv('test_authors.csv')
test_data.head()

,text
0,"-Да, я поторопился. Капитан, примите мои извин..."
1,-Похороны по первому разряду! Довольно благоро...
2,"Третий округ штата Мэн настолько велик, что ег..."
3,В мире существуют миллиарды и миллиарды богов....
4,Особенность историографии киевского периода со...


Заведем словарь соответствия имени автора его номеру:

In [ ]:
writers = ['Akunin', 'Bulychev', 'Chehov', 'Dostoevsky', 'Gogol', 'King',
       'Pratchett', 'Remark']
writers_to_label = {writer: i for i, writer in enumerate(writers)}
label_to_writers = {i: writer for i, writer in enumerate(writers)}

Создадим словарь dataset, который будет устроен так же, как переменная dataset в ноутбуке занятия. В тестовой части (dataset['test']) зададим для удобства всем текстам label=0.

In [ ]:
'''dataset = {}

dataset['train'] = [{'text':text, 'label':writers_to_label[label]} \
              for text, label in zip(np.array(train_data['text']), np.array(train_data['label']))]
dataset['test'] = [{'text':text, 'label': 0} \
              for text in np.array(test_data['text'])] '''

Ваша задача — обучить RNN-модель на тренировочных данных и получить максимальное возможное accuracy на тестовой части данных. За основу можно взять код с занятия. **Обратите внимание, что здесь у нас задача классификации на 8 классов, а не на 2, как было на занятии.**

Чтобы улучшить качество базовой модели, можно попробовать различные идеи экспериментов. Вот несколько идей:
* **Модель RNN**. Попробуйте LSTM и GRU. Мы советуем обратить внимание на [GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html), так как интерфейс этого класса ничем не отличается от обычной Vanilla RNN, которую мы использовали на семинаре.
* **Увеличение количества рекуррентных слоев модели**. Это можно сделать с помощью параметра `num_layers` в классе `nn.RNN`. В такой модели выходы первой RNN передаются в качестве входов второй RNN и так далее.
* **Изменение вида агрреграции выхода RNN**. В семинаре мы пробовали аггрегировать выходы RNN с помощью max, mean и получения последнего выхода. Можно попробовать другие варианты. Например, конкатенировать результат агрегации и эмбеддинг с последнего токена.
* **Подбор гиперпараметров и обучение до сходимости**. Можно, например, увеличить количество эпох обучения нейросети, а также попробовать различные гиперпараметры: размер словаря, `dropout_rate`, `hidden_dim`.

## Сдача задания

Ниже приведена функция, которую вам необходимо запустить для обученной модели, чтобы получить предсказания на тестовой выборке. Здесь model — ваша обученная модель, dataloader — test_dataloader, построенный на основе тестовой части данных (dataset['test']):

In [ ]:
'''def get_predictions(model, dataloader):
    """
    Calculate accuracy on data from dataloader.
    """

    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm_notebook(dataloader,
                                   desc=f'Evaluating'):
            logits = model(batch['input_ids'])
            predictions.append(logits.argmax(dim=1))

    predictions = torch.cat(predictions).data.cpu().numpy()

    return predictions
    '''

In [ ]:

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# --- Подготовка данных ---
train_data = pd.read_csv('train_authors.csv')
test_data = pd.read_csv('test_authors.csv')

writers = ['Akunin', 'Bulychev', 'Chehov', 'Dostoevsky', 'Gogol', 'King',
           'Pratchett', 'Remark']
writers_to_label = {writer: i for i, writer in enumerate(writers)}
label_to_writers = {i: writer for i, writer in enumerate(writers)}

# Предобработка данных (упрощенная)
def preprocess_text(text):
    return text.strip()

train_texts = [preprocess_text(text) for text in train_data['text']]
test_texts = [preprocess_text(text) for text in test_data['text']]
train_labels = [writers_to_label[author] for author in train_data['author']]


# --- Загрузка токенизатора BERT ---
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') # или другая модель BERT

# --- Создание датасета ---
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        return {
            'text': text,
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, [0] * len(test_texts), tokenizer)  # метки для теста не важны

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# --- Загрузка предобученной модели BERT ---
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(writers)).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5) # Измененная скорость обучения
# --- Обучение модели ---
epochs = 12

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')

# --- Предсказания ---
def get_predictions(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Predicting'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(pred)
    return predictions

predictions = get_predictions(model, test_loader)
predictions = [label_to_writers[x] for x in predictions]
np.save('submission_hw07.npy', np.array(predictions), allow_pickle=True)
print('Predictions saved to submission_hw07.npy')

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Epoch 11/12, Loss: 0.1013036899123375


Epoch 12/12:   0%|          | 0/109 [00:00<?, ?it/s]

Epoch 12/12, Loss: 0.08563154982402921


Predicting:   0%|          | 0/21 [00:00<?, ?it/s]

Predictions saved to submission_hw07.npy


In [ ]:

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
import pandas as pd
import re
import nltk

# Загрузка nltk (если не загружено)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Установите device для использования GPU, если доступно
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


# --- Подготовка данных ---
train_data = pd.read_csv('train_authors.csv')
test_data = pd.read_csv('test_authors.csv')


def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Удаляем пунктуацию
    return text.strip()


train_texts = [preprocess_text(text) for text in train_data['text']]
test_texts = [preprocess_text(text) for text in test_data['text']]
train_labels = train_data['author'].map({'Akunin': 0, 'Bulychev': 1, 'Chehov': 2, 'Dostoevsky': 3, 'Gogol': 4, 'King': 5, 'Pratchett': 6, 'Remark': 7}).values


# --- Загрузка токенизатора BERT ---
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


# --- Создание датасета ---
class BERTDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = BERTDataset(train_texts, train_labels, tokenizer)
test_dataset = BERTDataset(test_texts, [0] * len(test_texts), tokenizer)  # метки для теста не важны

# Важно для воспроизводимости
train_sampler = RandomSampler(train_dataset)
test_sampler = SequentialSampler(test_dataset)

train_loader = DataLoader(train_dataset, batch_size=16, sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_size=16, sampler=test_sampler)



# --- Загрузка и инициализация модели BERT ---
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=8).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)


# --- Функция для получения предсказаний ---
def get_predictions(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(pred)
    return predictions



# --- Обучение модели ---
epochs = 12
for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# --- Предсказания ---
predictions = get_predictions(model, test_loader)
predictions = [label_to_writers[x] for x in predictions]
np.save('submission_hw07.npy', np.array(predictions), allow_pickle=True)
print('Predictions saved to submission_hw07.npy')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/12:   0%|          | 0/109 [00:00<?, ?it/s]

Epoch 2/12:   0%|          | 0/109 [00:00<?, ?it/s]

...

Epoch 10/12:   0%|          | 0/109 [00:00<?, ?it/s]

Epoch 11/12:   0%|          | 0/109 [00:00<?, ?it/s]

Epoch 12/12:   0%|          | 0/109 [00:00<?, ?it/s]

Predicting:   0%|          | 0/21 [00:00<?, ?it/s]

Predictions saved to submission_hw07.npy
